In [2]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-du9q_sk3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-du9q_sk3
  Resolved https://github.com/huggingface/transformers.git to commit 102b5ff4a813eea848bb82ff2f451e0f6b17b30c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import transformers

In [4]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
import torch
import soundfile as sf
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")


In [5]:
# load xvector containing speaker's voice characteristics from a dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


## Mount Google Drive

In [10]:
import os
import pandas as pd

In [11]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Directories and DataFrame

In [12]:
data_dir = "/content/gdrive/MyDrive/data_sampled/"
data_dir_audio = os.path.join(data_dir, "cv-valid-test")

In [13]:
audio_csv = os.path.join(data_dir, "cv_valid_test_FINAL.csv")
df = pd.read_csv(audio_csv, index_col=0)

In [14]:
df["filename_wav"] = df["filename_wav"].apply(lambda x: os.path.join("/content/gdrive/MyDrive/", x))
df

,filename,text,up_votes,down_votes,age,gender,accent,duration,len_words,filename_wav
0,data_sampled//cv-valid-test/sample-000377.mp3,i'm never more aware of a room's acoustics tha...,2,0,NaN,NaN,NaN,NaN,22,/content/gdrive/MyDrive/data_sampled//cv-valid...
8,data_sampled//cv-valid-test/sample-001202.mp3,you don't have to be rich skinny popular or ev...,4,0,NaN,NaN,NaN,NaN,20,/content/gdrive/MyDrive/data_sampled//cv-valid...
9,data_sampled//cv-valid-test/sample-001276.mp3,it was only when i got this close to it that t...,3,0,fourties,male,us,NaN,21,/content/gdrive/MyDrive/data_sampled//cv-valid...
21,data_sampled//cv-valid-test/sample-002279.mp3,we miss you and miss having a friend like you ...,1,0,sixties,female,england,NaN,22,/content/gdrive/MyDrive/data_sampled//cv-valid...
34,data_sampled//cv-valid-test/sample-003965.mp3,a part of me is also trying to come to terms w...,1,0,NaN,NaN,NaN,NaN,24,/content/gdrive/MyDrive/data_sampled//cv-valid...


In [15]:
wav_files = list(df["filename_wav"])

In [24]:
dest_files = [f"t5-{x[-10:]}" for x in wav_files]
dest_files 

['t5-000377.wav',
 't5-001202.wav',
 't5-001276.wav',
 't5-002279.wav',
 't5-003965.wav']

In [25]:
texts = list(df["text"])

In [26]:
texts

["i'm never more aware of a room's acoustics than when i'm trying to enjoy a snack i have no intention of sharing",
 "you don't have to be rich skinny popular or even own a cape to perform a random act of kindness",
 'it was only when i got this close to it that the strangeness of it was at all evident to me',
 'we miss you and miss having a friend like you and i am so happy that you two got to catch up',
 "a part of me is also trying to come to terms with the fact that i won't be able to travel as much anymore"]

In [45]:
texts = [f"{text}." for text in texts]

In [46]:
dest_dir = "/content/gdrive/MyDrive/speech_tech_audios/T5/"

In [47]:
for i, text in enumerate(texts):
    print(text)
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

    ttle = dest_files[i]
    sound_file = os.path.join(dest_dir, ttle)
    sf.write(sound_file, speech.numpy(), samplerate=16000)

i'm never more aware of a room's acoustics than when i'm trying to enjoy a snack i have no intention of sharing.
you don't have to be rich skinny popular or even own a cape to perform a random act of kindness.
it was only when i got this close to it that the strangeness of it was at all evident to me.
we miss you and miss having a friend like you and i am so happy that you two got to catch up.
a part of me is also trying to come to terms with the fact that i won't be able to travel as much anymore.


In [52]:
from IPython.display import Audio, display
sound_file = os.path.join(dest_dir, dest_files[4])
display(Audio(sound_file, autoplay=True))